In [2]:
from pyspark.sql.types import *
%load_ext autoreload
%autoreload 2

import pyspark, os, sys
from delta import *
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *
from pyspark.sql.functions import col, lit, max as spark_max, length, row_number, explode, unix_timestamp, from_unixtime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/docker/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../docker/gcs-connector-hadoop.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [4]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
absolute_path_to_landing = '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma'

:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0665a6b0-59ad-4342-b0ee-347f87bc462c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 75ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [5]:
df = spark.read.format("delta").load(absolute_path_to_landing + "/delta_sports/matches")

In [6]:
df

25/05/29 18:34:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


fixture,goals,league,score,teams
{2025-04-25T00:00...,"{2, 0}","{USA, https://med...","{{NULL, NULL}, {2...","{{18814, https://..."
{2025-04-25T00:00...,"{1, 1}","{Ecuador, https:/...","{{NULL, NULL}, {1...","{{1159, https://m..."
{2025-04-25T00:00...,"{0, 1}","{World, NULL, 13,...","{{NULL, NULL}, {0...","{{1137, https://m..."
{2025-04-25T00:00...,"{0, 2}","{Mexico, https://...","{{NULL, NULL}, {0...","{{2299, https://m..."
{2025-04-25T00:30...,"{2, 2}","{World, NULL, 13,...","{{NULL, NULL}, {2...","{{2546, https://m..."
{2025-04-25T00:30...,"{1, 2}","{World, NULL, 11,...","{{NULL, NULL}, {1...","{{135, https://me..."
{2025-04-25T00:30...,"{2, 2}","{World, NULL, 11,...","{{NULL, NULL}, {2...","{{2562, https://m..."
{2025-04-25T00:35...,"{1, 2}","{Brazil, https://...","{{NULL, NULL}, {1...","{{134, https://me..."
{2025-04-25T01:00...,"{NULL, NULL}","{Barbados, https:...","{{NULL, NULL}, {N...","{{8406, https://m..."
{2025-04-25T01:00...,"{0, 2}","{Mexico, https://...","{{NULL, NULL}, {0...","{{14870, https://..."


In [23]:
table_subpath = 'delta_sports/matches'
loader = IncrementalLoader(spark, absolute_path_to_landing, table_subpath)
df = loader.get_new_data()
df.head(5)

INFO:dags.lib.pt_utils:Using CDF from version 13


25/05/22 15:51:20 WARN DeltaHistoryManager: Found Delta commit 0 with a timestamp 1745921644516 which is greater than the next commit timestamp 1745921644042.
25/05/22 15:51:20 WARN DeltaHistoryManager: Found Delta commit 1 with a timestamp 1745921644517 which is greater than the next commit timestamp 1745921643917.
25/05/22 15:51:20 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921644518 which is greater than the next commit timestamp 1745921642587.
25/05/22 15:51:20 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921644519 which is greater than the next commit timestamp 1745921644158.
25/05/22 15:51:20 WARN DeltaHistoryManager: Found Delta commit 4 with a timestamp 1745921644520 which is greater than the next commit timestamp 1745921642587.
25/05/22 15:51:20 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921644521 which is greater than the next commit timestamp 1745921642590.
25/05/22 15:51:20 WARN DeltaHistoryManager: Fo

[]

In [18]:
sproc = SportsMatchesProcessor(spark, df)
sproc.expand()
sproc.normalize_text(['referee', 'venue_city', 'venue_name', 'team_away_name', 'team_home_name', 'status_long'])

sproc.extract_teams()
sproc.extract_venues()
sproc.remove_useless_columns()
sproc.ensure_schema()
sproc.remove_clear_duplicates()
sproc.remove_hidden_duplicates(['fixture_id', 'status_long'])

+------+--------------------+----------+------------+-------------+--------------------+--------------+------------+--------------+------------+----------+--------+--------------------+--------+--------------------+------------+--------------------+--------------------+----------------+------------+--------------------+--------------------+----------------+----------+----------+
|league|        fixture_date|fixture_id|period_first|period_second|             referee|status_elapsed|status_extra|   status_long|status_short| timestamp|timezone|          venue_city|venue_id|          venue_name|team_away_id|      team_away_logo|      team_away_name|team_away_winner|team_home_id|      team_home_logo|      team_home_name|team_home_winner|goals_away|goals_home|
+------+--------------------+----------+------------+-------------+--------------------+--------------+------------+--------------+------------+----------+--------+--------------------+--------+--------------------+------------+--------

In [25]:
sproc = SportsMatchesProcessor(spark, df)
sproc.expand()
print(sproc.df.head(1))
sproc.normalize_text(['referee', 'venue_city', 'venue_name', 'team_away_name', 'team_home_name', 'status_long'])
print(sproc.df.head(1))

[Row(league=909, fixture_date='2025-04-25T00:00:00+00:00', fixture_id=1345365, period_first=1745539200, period_second=1745542800, referee='R. Homik', status_elapsed=90, status_extra=7, status_long='Match Finished', status_short='FT', timestamp=1745539200, timezone='UTC', venue_city='Commerce City, Colorado', venue_id=None, venue_name='Dick&amp;apos;s Sporting Goods Park', team_away_id=18814, team_away_logo='https://media.api-sports.io/football/teams/18814.png', team_away_name='Minnesota United II', team_away_winner=True, team_home_id=4102, team_home_logo='https://media.api-sports.io/football/teams/4102.png', team_home_name='Colorado Rapids II', team_home_winner=False, goals_away=2, goals_home=0)]
[Row(league=909, fixture_date='2025-04-25T00:00:00+00:00', fixture_id=1345365, period_first=1745539200, period_second=1745542800, referee=None, status_elapsed=90, status_extra=7, status_long=None, status_short='FT', timestamp=1745539200, timezone='UTC', venue_city=None, venue_id=None, venue_na

In [17]:
df.select('teams').head(1)

[Row(teams=Row(away=Row(id=18814, logo='https://media.api-sports.io/football/teams/18814.png', name='Minnesota United II', winner=True), home=Row(id=4102, logo='https://media.api-sports.io/football/teams/4102.png', name='Colorado Rapids II', winner=False)))]

In [20]:
sproc.teams.groupBy('team_id').count().filter('count>1')
sproc.venues.groupBy('venue_id').count().filter('count>1')


venue_id,count


In [22]:
sproc.merge_with_trusted("../data/letstalk_trusted_zone_bdma", 'delta_sports', ['fixture_id', 'status_long'])
loader.update_control_table()

INFO:dags.lib.pt_utils:Saving unique records from overlapping ones


--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 1150, in emit
    msg = self.format(record)
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 998, in format
    return fmt.format(record)
           ~~~~~~~~~~^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 711, in format
    record.message = record.getMessage()
                     ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 400, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as

INFO:dags.lib.pt_utils:Added new 0 unique records to table at path ../data/letstalk_trusted_zone_bdma/delta_sports/teams
INFO:dags.lib.pt_utils:Saving unique records from overlapping ones


--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 1150, in emit
    msg = self.format(record)
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 998, in format
    return fmt.format(record)
           ~~~~~~~~~~^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 711, in format
    record.message = record.getMessage()
                     ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 400, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as

INFO:dags.lib.pt_utils:Added new 0 unique records to table at path ../data/letstalk_trusted_zone_bdma/delta_sports/venues


--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 1150, in emit
    msg = self.format(record)
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 998, in format
    return fmt.format(record)
           ~~~~~~~~~~^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 711, in format
    record.message = record.getMessage()
                     ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 400, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as

INFO:dags.lib.pt_utils:Saving unique records from overlapping ones
INFO:dags.lib.pt_utils:Added new 0 unique records
INFO:dags.lib.pt_utils:Appending non overlapping records
INFO:dags.lib.pt_utils:Adding new 0 records


In [ ]:
full_distinct_df = sproc.df.distinct()
print(full_distinct_df.count())

partial_distinct_df = sproc.df.select('fixture_id', 'status_long').distinct()
print(partial_distinct_df.count())


In [ ]:
sproc.merge_with_trusted("../data/letstalk_trusted_zone_bdma", 'delta_sports', ['fixture_id', 'status_long'])


In [ ]:
sproc.merge_with_trusted("../data/letstalk_trusted_zone_bdma", 'delta_sports', ['fixture_id', 'status_long'])
